# 5. Evaluación final con `test_master` (enfoque purista)

En este notebook realizamos la **evaluación final** del modelo de churn valioso usando el conjunto de test que se reservó desde el principio (`test_master.csv`).

Pasos generales:

1. Cargar `train_master` y `test_master`.
2. Replicar la creación de variables del EDA (Recency, MntTotal, CLV, etc.).
3. Definir `Churn_Valioso_KMeans` en train y test usando las **mismas reglas**.
4. Preparar `X_test` e `y_test`.
5. Cargar el modelo `churn_pipeline.pkl` entrenado en el notebook 4.
6. Evaluar métricas finales (ROC-AUC, matriz de confusión, recall, F1).


## 1. Carga de `train_master` y `test_master`

Cargamos los archivos generados en el notebook 0. `train_master` se usa solo para obtener la mediana de `CLV_log` (para reproducir la definición de valioso).

In [ ]:
text = """import pandas as pd
from pathlib import Path

data_path = Path("../data")
train = pd.read_csv(data_path / "train_master.csv")
test = pd.read_csv(data_path / "test_master.csv")

train.shape, test.shape
"""
print(text)

## 2. Replicar la ingeniería de variables del EDA

Aquí debes replicar la lógica usada en `1_EDA_Superstore.ipynb` para crear:

- `MntTotal`
- `TotalPurchases`
- `Perc_WebPurchases`, `Perc_CatalogPurchases`, `Perc_StorePurchases`
- `CLV_simple`
- `CLV_log`

Las fórmulas dependen de los nombres de columnas de tu dataset original (por ejemplo, sumas de importes por canal y número de compras).

In [ ]:
text = """# Ejemplo genérico: ajusta estos nombres según tu dataset real
# train["MntTotal"] = train[["MntWines", "MntFruits", ...]].sum(axis=1)
# test["MntTotal"] = test[["MntWines", "MntFruits", ...]].sum(axis=1)

# train["TotalPurchases"] = train[["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases"]].sum(axis=1)
# test["TotalPurchases"] = test[["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases"]].sum(axis=1)

# train["Perc_WebPurchases"] = train["NumWebPurchases"] / train["TotalPurchases"].replace(0, 1)
# test["Perc_WebPurchases"] = test["NumWebPurchases"] / test["TotalPurchases"].replace(0, 1)

# train["Perc_CatalogPurchases"] = train["NumCatalogPurchases"] / train["TotalPurchases"].replace(0, 1)
# test["Perc_CatalogPurchases"] = test["NumCatalogPurchases"] / test["TotalPurchases"].replace(0, 1)

# train["Perc_StorePurchases"] = train["NumStorePurchases"] / train["TotalPurchases"].replace(0, 1)
# test["Perc_StorePurchases"] = test["NumStorePurchases"] / test["TotalPurchases"].replace(0, 1)

# train["CLV_simple"] = train["MntTotal"] * train["TotalPurchases"]
# test["CLV_simple"] = test["MntTotal"] * test["TotalPurchases"]

# import numpy as np
# train["CLV_log"] = np.log1p(train["MntTotal"]) * np.log1p(train["TotalPurchases"])
# test["CLV_log"] = np.log1p(test["MntTotal"]) * np.log1p(test["TotalPurchases"])
"""
print(text)

## 3. Definir `Churn_Valioso_KMeans` en train y test

Usamos las mismas reglas que en el EDA:

- `UMBRAL_RECENCY_KMEANS = 83`.
- Mediana de `CLV_log` calculada **solo con train**.

Luego definimos:

- `Churn_KMeans = 1` si `Recency >= 83`, si no 0.
- `Churn_Valioso_KMeans = 1` si (`Recency >= 83` y `CLV_log >= mediana_train`).

In [ ]:
text = """UMBRAL_RECENCY_KMEANS = 83

mediana_clv_train = train["CLV_log"].median()

for df_tmp in [train, test]:
    df_tmp["Churn_KMeans"] = (df_tmp["Recency"] >= UMBRAL_RECENCY_KMEANS).astype(int)
    df_tmp["Churn_Valioso_KMeans"] = (
        (df_tmp["Recency"] >= UMBRAL_RECENCY_KMEANS)
        & (df_tmp["CLV_log"] >= mediana_clv_train)
    ).astype(int)

test["Churn_Valioso_KMeans"].value_counts(normalize=True).round(4) * 100
"""
print(text)

## 4. Preparar `X_test` e `y_test`

Usamos exactamente las mismas columnas de features que en el notebook 4.

In [ ]:
text = """feature_cols = [
    "Recency", "MntTotal", "TotalPurchases", "Income",
    "Perc_CatalogPurchases", "NumWebVisitsMonth",
    "Kidhome", "Teenhome",
    "Education", "Marital_Status"
]
target_col = "Churn_Valioso_KMeans"

X_test = test[feature_cols].copy()
y_test = test[target_col].copy()
X_test.shape, y_test.shape
"""
print(text)

## 5. Cargar el modelo entrenado

Cargamos el pipeline `churn_pipeline.pkl` guardado en la carpeta `models/` por el notebook 4.

In [ ]:
text = """import joblib
from pathlib import Path

models_path = Path("../models")
churn_model = joblib.load(models_path / "churn_pipeline.pkl")
churn_model
"""
print(text)

## 6. Evaluación final en test

Calculamos las métricas en el conjunto de test (`X_test`, `y_test`):

- ROC-AUC
- Matriz de confusión
- Precision, Recall, F1

Esto nos da una estimación honesta del rendimiento del modelo en datos nuevos.

In [ ]:
text = """from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

y_test_proba = churn_model.predict_proba(X_test)[:, 1]
y_test_pred = churn_model.predict(X_test)

roc_test = roc_auc_score(y_test, y_test_proba)
print(f"ROC-AUC (test): {roc_test:.4f}")

print("\nMatriz de confusión (test):")
print(confusion_matrix(y_test, y_test_pred))

print("\nReporte de clasificación (test):")
print(classification_report(y_test, y_test_pred, digits=4))
"""
print(text)

## 7. Conclusiones finales

Aquí debes redactar tus conclusiones sobre:

- Rendimiento del modelo en test (¿mantiene lo visto en validación?).
- Capacidad para detectar churn valioso (Recall de la clase 1).
- Posibles mejoras futuras (más features, otros modelos, calibración de probabilidades, etc.).
